In [ ]:
#import packages necessary to create basic function for Kernel Perceptron
import numpy as np
import pandas as pd
import random 

In [ ]:
#create a function that allow to shift from a positive value to 1 and a negative value to -1. Used in perceptron algorithm
def sign(value):
    if value > 0:
        fvalue = 1
    elif value <= 0:
        fvalue = -1
    return fvalue

In [ ]:
#define function to compute the gram matrix, necessary to output the predictor from the learning algorithm
def gram_poly(A, d):
    
    return np.float32((1 + np.matmul(A, A.T))**d)

In [ ]:
#define function to compute the Kernel Matrix, necessary to make prediction given a predictor
def kernel_poly(A, B, d):
    
    return np.float32((1 + np.matmul(A, B.T))**d)

In [ ]:
#extract a predictor (one vs all) at the end of each epochs
def find_predictors(x_train, y_train, kernel, epochs, row):
    saved_predictors = []
    alpha = np.zeros((len(x_train)))
    for epoch in range(epochs):
        for i in range(len(x_train)):
            val= np.dot((alpha * y_train[:,row]),kernel[i,:])
            val = sign(val)
            if val != y_train[i, row]:
                alpha[i] = alpha[i] + 1
        predictor = alpha.copy()
        saved_predictors.append(predictor)
    return saved_predictors

In [ ]:
#function to take a binary final predictors at the end of each epoch and make the mean of each binary predictor.
def mean_classifier(classifiers_list):
    mean_classifier = []
    for classifier in classifiers_list:
        classifier_matrix = np.array(classifier)
        classifier_matrix = classifier_matrix.mean(axis = 0)
        mean_classifier.append(classifier_matrix)
    mean_classifier = np.array(mean_classifier)
    return(mean_classifier)

In [ ]:
#from the set of a binary predictor (example: 2)
#extract the one which minimize the training error for prediction of the number 2
def define_bests(x_train, y_train, gram_matrix, predictors):
    best_for_binary = []
    for index, list_ in enumerate(predictors):
        value_error = []
        for ind, predictor in enumerate(list_):
            wrong = 0
            for i in range(len(x_train)):
                s = np.dot((predictor*y_train[:,index]),gram_matrix[i,:])
                s = sign(s)
                if y_train[i,index] != s:
                    wrong += 1 
            value = (wrong / len(x_train))*100
            value_error.append(value)
        value_error = np.array(value_error)
        index_min = np.argmin(value_error)
        best = list_[index_min]
        best_for_binary.append(best)
    return(best_for_binary) 

In [ ]:
#function to compute the accuracy of a predictor
def accuracy_test(y_train, y_test, multi_best, kernel_matrix, x_train):
    num_correct = 0
    for i in range(len(y_test)):
        predict = np.zeros(len(multi_best))
        for j in range(len(multi_best)):
                predict[j] =  np.dot((multi_best[j,:]*y_train[:, j]),kernel_matrix[i,:])
        prediction = np.argmax(predict)
        correct = np.argmax(y_test[i,:])
        if prediction == correct:
            num_correct += 1
    percentage = (num_correct/ len(y_test))*100
    return percentage

In [ ]:
#function that given the prediction and the real values, output the confusion matrix
def confusion_matrix(y_train, y_test, multi_best, kernel_matrix, x_train):
    num_correct = 0
    conf_matrix = np.zeros(shape=(y_train.shape[1], y_train.shape[1]))
    for i in range(len(y_test)):
        predict = np.zeros(len(multi_best))
        for j in range(len(multi_best)):
                predict[j] =  np.dot((multi_best[j,:]*y_train[:, j]),kernel_matrix[i,:])
        prediction = np.argmax(predict)
        correct = np.argmax(y_test[i,:])
        conf_matrix[correct, prediction] += 1
    return conf_matrix

In [ ]:
#function to compute the training accuracy
def accuracy_train(y_train, x_train, multi_best, kernel_matrix):
    num_correct = 0
    for i in range(len(y_train)):
        predict = np.zeros(len(multi_best))
        for j in range(len(multi_best)):
            predict[j] = np.dot((multi_best[j,:]*y_train[:, j]),kernel_matrix[i,:])
        prediction = np.argmax(predict)
        correct = np.argmax(y_train[i,:])
        if prediction == correct:
            num_correct +=  1
    percentage = (num_correct/ len(y_train))*100
    return percentage

In [ ]:
#main function: given a training set and test set, output the training accuracy and the test accuracy 
#for both the mean_predictor and the best_predictor
def kernel_perceptron(x_train, x_test, y_train, y_test, dimensions, epochs):
    gram = gram_poly(x_train, dimensions)
    gram = np.float32(gram)
    One_vs_all_predictors = []
    for binary_classificators in range(y_train.shape[1]):
        predictor = find_predictors(x_train, y_train, gram, epochs, binary_classificators) 
        One_vs_all_predictors.append(predictor)
    kernel_test = kernel_poly(x_test, x_train, dimensions)
    kernel_test = np.float32(kernel_test)
    final_best_classificator = define_bests(x_train, y_train, gram, One_vs_all_predictors)
    final_best_classificator = np.array(final_best_classificator)
    mean_classificator = mean_classifier(One_vs_all_predictors)
    result = []
    n_test = accuracy_test(y_train, y_test, final_best_classificator, kernel_test, x_train)
    result.append(n_test)
    n_train = accuracy_train(y_train, x_train, final_best_classificator, gram)
    result.append(n_train)
    n_test = accuracy_test(y_train, y_test, mean_classificator, kernel_test, x_train)
    result.append(n_test)
    n_train = accuracy_train(y_train, x_train, mean_classificator, gram)
    result.append(n_train)
    conf_matrix_best = confusion_matrix(y_train, y_test, final_best_classificator, kernel_test, x_train)
    result.append(conf_matrix_best)
    conf_matrix_mean = confusion_matrix(y_train, y_test, mean_classificator, kernel_test, x_train)
    result.append(conf_matrix_mean)
    return(result)

In [ ]:
#function used to compute true positive, false negative, true positive e true negative and 
#consequently the precision and the recall, that are measures that could be very usefull
def metrics(conf_matrix):
    TP = np.diag(conf_matrix)
    FP = np.sum(conf_matrix, axis=0) - TP
    FN = np.sum(conf_matrix, axis=1) - TP
    num_classes = 10
    TN = []
    for i in range(num_classes):
        temp = np.delete(conf_matrix, i, 0)    # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TN.append(sum(sum(temp)))
    metric = []
    precision = TP/(TP+FP)
    metric.append(precision)
    recall = TP/(TP+FN)
    metric.append(recall)
    return(metric)